<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/GAN/CGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional GAN

DCGAN 에서는 이미지가 임의로 생성되며, 생성기에서 어떤 숫자를 생성할지는 제어할 수 없다. 이 문제를 원-핫 벡터를 추가로 입력을 받는 것으로 해결한 것이 바로 Conditional GAN (CGAN) 이다. 

동일한 GAN의 구조를 사용하면서도 생성기에서 원-핫 벡터를 추가로 받고 새로운 Dense 계층을 추가해 잠재벡터와 연결된다. 또, 판별기에서는 새로운 Dense 계층이 추가된다. 이 새로운 게층은 원-핫 벡터를 처리하고 뒤따라나오는 CNN 계층의 다른 입력과 연결하기에 적합한 형상으로 변경하기 위해 사용된다.

생성기는 100차원 입력 벡터와 특정 숫자로 부터 가짜 이미지를 생성하는 것을 학습하며, 판별기는 진짜와 가짜 이미지, 그리고 그에 대응하는 레이블을 기반으로 진짜와 가짜 이미지를 분류한다.

CGAN 의 기본 원리는 GAN 과 동일하지만,  판별기와 생성기 입력에 원-핫 레이블 y 라는 조건이 부여된다는 점이 다르다.

판별기와 생성기는 아래와 같이 구성된다.

# 판별기 모델 구성 함수 정의

In [ ]:
def build_discriminator(inputs, y_labels, image_size):

# 생성기 모델 구성 함수 정의